In [1]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
# from data_loader import DataLoader
import numpy as np
import os

c:\users\mooney\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import scipy
from glob import glob
# import numpy as np

class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

        batch = np.random.choice(path, size=batch_size)

        imgs_A, imgs_B = [], []
        for img in batch:
            img = self.imread(img)
            h, w, _ = img.shape
            half_w = int(w/2)
            img_A = img[:, :half_w, :]
            img_B = img[:, half_w:, :]

            img_A = scipy.misc.imresize(img_A, self.img_res)
            img_B = scipy.misc.imresize(img_B, self.img_res)

            if not is_testing and np.random.random() > 0.5:
                    img_A = np.fliplr(img_A)
                    img_B = np.fliplr(img_B)

            imgs_A.append(img_A)
            imgs_B.append(img_B)

        imgs_A = np.array(imgs_A)/127.5 - 1.
        imgs_B = np.array(imgs_B)/127.5 - 1.

        return imgs_A, imgs_B

    def load_batch(self, batch_size=1, is_testing=False):
        data_type = "train" if not is_testing else "val"
        path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

        self.n_batches = int(len(path) / batch_size)

        for i in range(self.n_batches-1):
            batch = path[i*batch_size:(i+1)*batch_size]
            imgs_A, imgs_B = [], []
            for img in batch:
                img = self.imread(img)
                h, w, _ = img.shape
                half_w = int(w/2)
                img_A = img[:, :half_w, :]
                img_B = img[:, half_w:, :]

                img_A = scipy.misc.imresize(img_A, self.img_res)
                img_B = scipy.misc.imresize(img_B, self.img_res)

                if not is_testing and np.random.random() > 0.5:
                        img_A = np.fliplr(img_A)
                        img_B = np.fliplr(img_B)

                imgs_A.append(img_A)
                imgs_B.append(img_B)

            imgs_A = np.array(imgs_A)/127.5 - 1.
            imgs_B = np.array(imgs_B)/127.5 - 1.

            yield imgs_A, imgs_B

    def load_img(self, path):
        img = self.imread(path)
        img = scipy.misc.imresize(img, self.img_res)
        img = img/127.5 - 1.
        return img[np.newaxis, :, :, :]

    def imread(self, path):

        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [3]:
from keras.callbacks import ModelCheckpoint # callback checkpoint

MODEL_SAVE_FOLDER_PATH = './saved_model/'
if not os.path.exists(MODEL_SAVE_FOLDER_PATH):
    os.mkdir(MODEL_SAVE_FOLDER_PATH)
    

In [ ]:
class DiscoGAN():
    def __init__(self):
        # Input shape
        self.img_rows = 128
        self.img_cols = 128
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)

        # Configure data loader
        self.dataset_name = 'edges2shoes'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))


        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2**4)
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminators
        self.d_A = self.build_discriminator()
        self.d_B = self.build_discriminator()
        self.d_A.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])
        self.d_B.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        #-------------------------
        # Construct Computational
        #   Graph of Generators
        #-------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        # Input images from both domains
        img_A = Input(shape=self.img_shape)
        img_B = Input(shape=self.img_shape)

        # Translate images to the other domain
        fake_B = self.g_AB(img_A)
        fake_A = self.g_BA(img_B)
        # Translate images back to original domain
        reconstr_A = self.g_BA(fake_B)
        reconstr_B = self.g_AB(fake_A)

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        # Objectives
        # + Adversarial: Fool domain discriminators
        # + Translation: Minimize MAE between e.g. fake B and true B
        # + Cycle-consistency: Minimize MAE between reconstructed images and original
        self.combined = Model(inputs=[img_A, img_B],
                              outputs=[ valid_A, valid_B,
                                        fake_B, fake_A,
                                        reconstr_A, reconstr_B ])
        self.combined.compile(loss=['mse', 'mse',
                                    'mae', 'mae',
                                    'mae', 'mae'],
                              optimizer=optimizer)

    def build_generator(self):
        """U-Net Generator"""

        def conv2d(layer_input, filters, f_size=4, normalize=True):
            """Layers used during downsampling"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalize:
                d = InstanceNormalization()(d)
            return d

        def deconv2d(layer_input, skip_input, filters, f_size=4, dropout_rate=0):
            """Layers used during upsampling"""
            u = UpSampling2D(size=2)(layer_input)
            u = Conv2D(filters, kernel_size=f_size, strides=1, padding='same', activation='relu')(u)
            if dropout_rate:
                u = Dropout(dropout_rate)(u)
            u = InstanceNormalization()(u)
            u = Concatenate()([u, skip_input])
            return u

        # Image input
        d0 = Input(shape=self.img_shape)

        # Downsampling
        d1 = conv2d(d0, self.gf, normalize=False)
        d2 = conv2d(d1, self.gf*2)
        d3 = conv2d(d2, self.gf*4)
        d4 = conv2d(d3, self.gf*8)
        d5 = conv2d(d4, self.gf*8)
        d6 = conv2d(d5, self.gf*8)
        d7 = conv2d(d6, self.gf*8)

        # Upsampling
        u1 = deconv2d(d7, d6, self.gf*8)
        u2 = deconv2d(u1, d5, self.gf*8)
        u3 = deconv2d(u2, d4, self.gf*8)
        u4 = deconv2d(u3, d3, self.gf*4)
        u5 = deconv2d(u4, d2, self.gf*2)
        u6 = deconv2d(u5, d1, self.gf)

        u7 = UpSampling2D(size=2)(u6)
        output_img = Conv2D(self.channels, kernel_size=4, strides=1,
                            padding='same', activation='tanh')(u7)

        return Model(d0, output_img)

    def build_discriminator(self):

        def d_layer(layer_input, filters, f_size=4, normalization=True):
            """Discriminator layer"""
            d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
            d = LeakyReLU(alpha=0.2)(d)
            if normalization:
                d = InstanceNormalization()(d)
            return d

        img = Input(shape=self.img_shape)

        d1 = d_layer(img, self.df, normalization=False)
        d2 = d_layer(d1, self.df*2)
        d3 = d_layer(d2, self.df*4)
        d4 = d_layer(d3, self.df*8)

        validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d4)

        return Model(img, validity)

    ## gan 모델 저장할때는 제너레이터와 디스크리미네이터, gan 을 따로 저장
    def save_models(self, epoch):
        self.d_A.save("DiscoGAN_discriminatorA_{}_{}.hdf5".format(self.dataset_name, epoch))
        self.d_B.save("DiscoGAN_discriminatorB_{}_{}.hdf5".format(self.dataset_name, epoch))
        self.g_AB.save("DiscoGAN_generatorAB_{}_{}.hdf5".format(self.dataset_name, epoch))
        self.g_BA.save("DiscoGAN_generatorBA_{}_{}.hdf5".format(self.dataset_name, epoch))

        self.combined.save("DiscoGAN_combined_{}_{}.hdf5".format(self.dataset_name, epoch))
        
#         ModelCheckpoint(MODEL_SAVE_FOLDER_PATH, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    
    def train(self, epochs, batch_size=128, sample_interval=50):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(epochs):

            for batch_i, (imgs_A, imgs_B) in enumerate(self.data_loader.load_batch(batch_size)):

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                fake_B = self.g_AB.predict(imgs_A)
                fake_A = self.g_BA.predict(imgs_B)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                # Train the generators
                g_loss = self.combined.train_on_batch([imgs_A, imgs_B], [valid, valid, \
                                                                         imgs_B, imgs_A, \
                                                                         imgs_A, imgs_B])

                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[%d] [%d/%d] time: %s, [d_loss: %f, g_loss: %f]" % (epoch, batch_i,
                                                                        self.data_loader.n_batches,
                                                                        elapsed_time,
                                                                        d_loss[0], g_loss[0]))

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
                    
                    ## gan 모델 저장할때는 제너레이터와 디스크리미네이터, gan 을 따로 저장
                    self.save_models(epoch)
                    
    def sample_images(self, epoch, batch_i):
        os.makedirs('images/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A, imgs_B = self.data_loader.load_data(batch_size=1, is_testing=True)

        # Translate images to the other domain
        fake_B = self.g_AB.predict(imgs_A)
        fake_A = self.g_BA.predict(imgs_B)
        # Translate back to original domain
        reconstr_A = self.g_BA.predict(fake_B)
        reconstr_B = self.g_AB.predict(fake_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()
    


# if __name__ == '__main__':
#     gan = DiscoGAN()
#     gan.train(epochs=20, batch_size=1, sample_interval=200)
    
    

In [ ]:
# 트레이닝
gan = DiscoGAN()
gan.train(epochs=20, batch_size=1, sample_interval=200)

c:\users\mooney\appdata\local\programs\python\python35\lib\site-packages\scipy\misc\pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if issubdtype(ts, int):
c:\users\mooney\appdata\local\programs\python\python35\lib\site-packages\scipy\misc\pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):
c:\users\mooney\appdata\local\programs\python\python35\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


[0] [0/49825] time: 0:01:24.191315, [d_loss: 40.064262, g_loss: 88.915352]
[0] [1/49825] time: 0:01:53.659238, [d_loss: 17.945721, g_loss: 62.588535]
[0] [2/49825] time: 0:01:54.662609, [d_loss: 18.208439, g_loss: 2.375362]
[0] [3/49825] time: 0:01:55.668496, [d_loss: 1.705872, g_loss: 4.170474]
[0] [4/49825] time: 0:01:56.666496, [d_loss: 0.544836, g_loss: 3.867285]
[0] [5/49825] time: 0:01:57.670848, [d_loss: 1.237025, g_loss: 1.063127]
[0] [6/49825] time: 0:01:58.669962, [d_loss: 0.552942, g_loss: 3.780980]
[0] [7/49825] time: 0:01:59.662456, [d_loss: 0.779928, g_loss: 2.032652]
[0] [8/49825] time: 0:02:00.671819, [d_loss: 0.563958, g_loss: 2.084669]
[0] [9/49825] time: 0:02:01.677843, [d_loss: 0.481327, g_loss: 2.034203]
[0] [10/49825] time: 0:02:02.680756, [d_loss: 0.452190, g_loss: 2.727330]
[0] [11/49825] time: 0:02:03.677937, [d_loss: 0.537193, g_loss: 1.788063]
[0] [12/49825] time: 0:02:04.681389, [d_loss: 0.551528, g_loss: 1.827624]
[0] [13/49825] time: 0:02:05.678303, [d_los

[0] [111/49825] time: 0:03:44.160097, [d_loss: 0.271789, g_loss: 2.629143]
[0] [112/49825] time: 0:03:45.162511, [d_loss: 0.239652, g_loss: 2.709748]
[0] [113/49825] time: 0:03:46.167451, [d_loss: 0.264006, g_loss: 2.491544]
[0] [114/49825] time: 0:03:47.183713, [d_loss: 0.399355, g_loss: 2.214998]
[0] [115/49825] time: 0:03:48.190590, [d_loss: 0.271209, g_loss: 2.324560]
[0] [116/49825] time: 0:03:49.195033, [d_loss: 0.252763, g_loss: 2.853220]
[0] [117/49825] time: 0:03:50.202410, [d_loss: 0.254509, g_loss: 2.696871]
[0] [118/49825] time: 0:03:51.199327, [d_loss: 0.654354, g_loss: 1.798970]
[0] [119/49825] time: 0:03:52.194803, [d_loss: 0.445302, g_loss: 1.886072]
[0] [120/49825] time: 0:03:53.189776, [d_loss: 0.321017, g_loss: 1.958169]
[0] [121/49825] time: 0:03:54.200175, [d_loss: 0.294679, g_loss: 2.667527]
[0] [122/49825] time: 0:03:55.201595, [d_loss: 0.532897, g_loss: 1.666299]
[0] [123/49825] time: 0:03:56.200993, [d_loss: 0.290220, g_loss: 2.015097]
[0] [124/49825] time: 0:0

[0] [221/49825] time: 0:05:40.474557, [d_loss: 0.417400, g_loss: 1.802477]
[0] [222/49825] time: 0:05:41.467545, [d_loss: 0.417697, g_loss: 2.171996]
[0] [223/49825] time: 0:05:42.485337, [d_loss: 0.413728, g_loss: 2.002383]
[0] [224/49825] time: 0:05:43.524492, [d_loss: 0.486602, g_loss: 1.723935]
[0] [225/49825] time: 0:05:44.551677, [d_loss: 0.376617, g_loss: 2.546437]
[0] [226/49825] time: 0:05:45.557565, [d_loss: 0.393948, g_loss: 2.368627]
[0] [227/49825] time: 0:05:46.583790, [d_loss: 0.367458, g_loss: 2.414144]
[0] [228/49825] time: 0:05:47.584719, [d_loss: 0.318206, g_loss: 2.217025]
[0] [229/49825] time: 0:05:48.601019, [d_loss: 0.369601, g_loss: 2.169008]
[0] [230/49825] time: 0:05:49.647580, [d_loss: 0.408538, g_loss: 2.061080]
[0] [231/49825] time: 0:05:50.676335, [d_loss: 0.350918, g_loss: 2.269990]
[0] [232/49825] time: 0:05:51.675228, [d_loss: 0.435472, g_loss: 2.229713]
[0] [233/49825] time: 0:05:52.677690, [d_loss: 0.628971, g_loss: 1.736205]
[0] [234/49825] time: 0:0

[0] [331/49825] time: 0:07:31.864220, [d_loss: 0.426790, g_loss: 1.704151]
[0] [332/49825] time: 0:07:32.851301, [d_loss: 0.301759, g_loss: 2.380395]
[0] [333/49825] time: 0:07:33.855214, [d_loss: 0.224064, g_loss: 2.192306]
[0] [334/49825] time: 0:07:34.859071, [d_loss: 0.327448, g_loss: 2.178751]
[0] [335/49825] time: 0:07:35.867935, [d_loss: 0.328160, g_loss: 2.087360]
[0] [336/49825] time: 0:07:36.877290, [d_loss: 0.335607, g_loss: 2.349660]
[0] [337/49825] time: 0:07:37.881199, [d_loss: 0.278096, g_loss: 2.368409]
[0] [338/49825] time: 0:07:38.878701, [d_loss: 0.267216, g_loss: 2.188363]
[0] [339/49825] time: 0:07:39.896493, [d_loss: 0.254216, g_loss: 2.175107]
[0] [340/49825] time: 0:07:40.901342, [d_loss: 0.330220, g_loss: 1.834249]
[0] [341/49825] time: 0:07:41.907729, [d_loss: 0.210376, g_loss: 2.437575]
[0] [342/49825] time: 0:07:42.909690, [d_loss: 0.381072, g_loss: 1.630780]
[0] [343/49825] time: 0:07:43.910104, [d_loss: 0.295934, g_loss: 2.019467]
[0] [344/49825] time: 0:0

...

[0] [4555/49825] time: 1:20:22.419919, [d_loss: 0.274236, g_loss: 1.054363]
[0] [4556/49825] time: 1:20:23.428292, [d_loss: 0.266079, g_loss: 1.162688]
[0] [4557/49825] time: 1:20:24.425781, [d_loss: 0.289648, g_loss: 1.228697]
[0] [4558/49825] time: 1:20:25.436099, [d_loss: 0.250910, g_loss: 1.049447]
[0] [4559/49825] time: 1:20:26.439009, [d_loss: 0.262829, g_loss: 1.465316]
[0] [4560/49825] time: 1:20:27.435521, [d_loss: 0.253017, g_loss: 1.245470]
[0] [4561/49825] time: 1:20:28.440418, [d_loss: 0.276909, g_loss: 1.018057]
[0] [4562/49825] time: 1:20:29.441298, [d_loss: 0.270857, g_loss: 1.188165]
[0] [4563/49825] time: 1:20:30.438803, [d_loss: 0.262155, g_loss: 1.050756]
[0] [4564/49825] time: 1:20:31.453074, [d_loss: 0.279121, g_loss: 1.034447]
[0] [4565/49825] time: 1:20:32.464966, [d_loss: 0.273258, g_loss: 1.034518]
[0] [4566/49825] time: 1:20:33.466838, [d_loss: 0.258114, g_loss: 1.077541]
[0] [4567/49825] time: 1:20:34.469748, [d_loss: 0.268883, g_loss: 0.890232]
[0] [4568/49

[0] [4663/49825] time: 1:22:16.233588, [d_loss: 0.220611, g_loss: 0.986729]
[0] [4664/49825] time: 1:22:17.243447, [d_loss: 0.250823, g_loss: 1.456938]
[0] [4665/49825] time: 1:22:18.246901, [d_loss: 0.263474, g_loss: 0.848419]
[0] [4666/49825] time: 1:22:19.252246, [d_loss: 0.243210, g_loss: 1.205389]
[0] [4667/49825] time: 1:22:20.254663, [d_loss: 0.250386, g_loss: 1.164969]
[0] [4668/49825] time: 1:22:21.247158, [d_loss: 0.257928, g_loss: 1.077642]
[0] [4669/49825] time: 1:22:22.246600, [d_loss: 0.259821, g_loss: 0.835010]
[0] [4670/49825] time: 1:22:23.245591, [d_loss: 0.253267, g_loss: 1.034719]
[0] [4671/49825] time: 1:22:24.245482, [d_loss: 0.240983, g_loss: 1.154638]
[0] [4672/49825] time: 1:22:25.252854, [d_loss: 0.229087, g_loss: 1.222180]
[0] [4673/49825] time: 1:22:26.253321, [d_loss: 0.255986, g_loss: 1.024677]
[0] [4674/49825] time: 1:22:27.243801, [d_loss: 0.295195, g_loss: 0.978014]
[0] [4675/49825] time: 1:22:28.246713, [d_loss: 0.285921, g_loss: 0.792550]
[0] [4676/49

[0] [4771/49825] time: 1:24:05.031991, [d_loss: 0.247944, g_loss: 1.005059]
[0] [4772/49825] time: 1:24:06.030938, [d_loss: 0.268333, g_loss: 0.884951]
[0] [4773/49825] time: 1:24:07.038311, [d_loss: 0.223225, g_loss: 1.240878]
[0] [4774/49825] time: 1:24:08.036808, [d_loss: 0.262115, g_loss: 1.052573]
[0] [4775/49825] time: 1:24:09.038186, [d_loss: 0.305781, g_loss: 0.747728]
[0] [4776/49825] time: 1:24:10.031721, [d_loss: 0.262358, g_loss: 0.908437]
[0] [4777/49825] time: 1:24:11.044553, [d_loss: 0.265185, g_loss: 0.926359]
[0] [4778/49825] time: 1:24:12.050441, [d_loss: 0.262362, g_loss: 1.295407]
[0] [4779/49825] time: 1:24:13.056284, [d_loss: 0.249223, g_loss: 0.993077]
[0] [4780/49825] time: 1:24:14.056216, [d_loss: 0.287481, g_loss: 0.745873]
[0] [4781/49825] time: 1:24:15.069051, [d_loss: 0.266778, g_loss: 0.815074]
[0] [4782/49825] time: 1:24:16.074443, [d_loss: 0.288413, g_loss: 0.737056]
[0] [4783/49825] time: 1:24:17.074877, [d_loss: 0.267990, g_loss: 0.733060]
[0] [4784/49

[0] [4879/49825] time: 1:25:58.854095, [d_loss: 0.239206, g_loss: 1.009341]
[0] [4880/49825] time: 1:25:59.859980, [d_loss: 0.254127, g_loss: 1.049919]
[0] [4881/49825] time: 1:26:00.865420, [d_loss: 0.235635, g_loss: 1.048751]
[0] [4882/49825] time: 1:26:01.865311, [d_loss: 0.235754, g_loss: 1.344832]
[0] [4883/49825] time: 1:26:02.859838, [d_loss: 0.237387, g_loss: 0.966993]
[0] [4884/49825] time: 1:26:03.861213, [d_loss: 0.252251, g_loss: 1.133001]
[0] [4885/49825] time: 1:26:04.865156, [d_loss: 0.229199, g_loss: 1.153120]
[0] [4886/49825] time: 1:26:05.870012, [d_loss: 0.207999, g_loss: 1.376719]
[0] [4887/49825] time: 1:26:06.870943, [d_loss: 0.225355, g_loss: 1.075011]
[0] [4888/49825] time: 1:26:07.870381, [d_loss: 0.293951, g_loss: 1.168826]
[0] [4889/49825] time: 1:26:08.879790, [d_loss: 0.263206, g_loss: 1.416048]
[0] [4890/49825] time: 1:26:09.884637, [d_loss: 0.263516, g_loss: 1.095423]
[0] [4891/49825] time: 1:26:10.897472, [d_loss: 0.281754, g_loss: 0.951585]
[0] [4892/49

[0] [4987/49825] time: 1:27:47.420216, [d_loss: 0.290738, g_loss: 0.727804]
[0] [4988/49825] time: 1:27:48.426599, [d_loss: 0.282980, g_loss: 0.793081]
[0] [4989/49825] time: 1:27:49.432040, [d_loss: 0.276166, g_loss: 0.765962]
[0] [4990/49825] time: 1:27:50.440902, [d_loss: 0.282146, g_loss: 0.811946]
[0] [4991/49825] time: 1:27:51.446248, [d_loss: 0.277607, g_loss: 0.682813]
[0] [4992/49825] time: 1:27:52.456105, [d_loss: 0.275812, g_loss: 0.682154]
[0] [4993/49825] time: 1:27:53.451575, [d_loss: 0.274089, g_loss: 0.636351]
[0] [4994/49825] time: 1:27:54.459644, [d_loss: 0.275100, g_loss: 0.673778]
[0] [4995/49825] time: 1:27:55.480083, [d_loss: 0.270435, g_loss: 0.686362]
[0] [4996/49825] time: 1:27:56.482958, [d_loss: 0.267365, g_loss: 0.696569]
[0] [4997/49825] time: 1:27:57.478725, [d_loss: 0.261602, g_loss: 1.174627]
[0] [4998/49825] time: 1:27:58.485111, [d_loss: 0.223943, g_loss: 0.977812]
[0] [4999/49825] time: 1:27:59.494518, [d_loss: 0.263264, g_loss: 0.670465]
[0] [5000/49